# NN 2.0

## Defining data functions

Import needed libraries

In [1]:
# imports
import pandas as pd
import tensorflow as tf
import tensorflow.keras as ks
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

Get data from file and remove columns with text

In [2]:
def get_swissvotes_data(now:int=651, since:int=299)->pd.DataFrame:
    import re
    
    dataset = pd.read_csv("../data/formatted/swissvotes_edited_nov_2021.csv", sep=';')
    
    regex = re.compile("pdev_.*")
    to_excl = list(filter(regex.match, dataset.columns))
    
    dataset.drop(columns=to_excl, inplace=True)
    dataset.drop(columns=["legisjahr"], inplace=True)
    dataset.drop(columns=["titel_kurz_d", "titel_kurz_f", "titel_off_d", "titel_off_f", "stichwort"], inplace=True)
    dataset.drop(columns=["swissvoteslink", "anzahl", "anneepolitique", "bkchrono_de", "bkchrono_fr"], inplace=True)
    dataset.drop(columns=["curiavista_de", "curiavista_fr", "urheber", "bkresults_de", "bkresults_fr"], inplace=True)
    dataset.drop(columns=["bfsmap_de", "bfsmap_fr", "nach_cockpit_d", "nach_cockpit_f", "nach_cockpit_e"], inplace=True)
    dataset = dataset[dataset["anr"] < now] # we don't care about future votes
    dataset = dataset[dataset["anr"] >= since]
    
    return dataset
print(f"Defined {get_swissvotes_data}")

Defined <function get_swissvotes_data at 0x000002BA822F0D30>


In [3]:
def get_left_right_data(now:int=651, since:int=299)->pd.DataFrame:
    col_names = ["anr", "datum", "titel_kurz_d", "ja_proz", "links_rechts", "kons_prog"]
    dataset = pd.read_csv("../data/formatted/brj_lire_konslib_edited.csv", sep=',')
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) < now]
    dataset = dataset.replace("#ZAHL!", np.nan).replace("Nan", np.nan)
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) >= since]
    
    return pd.DataFrame(dataset.drop(columns=["titel_kurz"]), index=get_swissvotes_data(now=now, since=since).index)

print(f"Defined {get_left_right_data}")

Defined <function get_left_right_data at 0x000002BA82309040>


In [4]:
def get_l_r_onehot(data:pd.DataFrame=get_left_right_data()["li_re"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["l3", "l2", "l1", "nlr", "r1", "r2", "r3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_l_r_onehot}")

Defined <function get_l_r_onehot at 0x000002BA82309670>


In [5]:
def get_kons_prog_onehot(data:pd.DataFrame=get_left_right_data()["kons_prog"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["k3", "k2", "k1", "nkp", "p1", "p2", "p3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_kons_prog_onehot}")

Defined <function get_kons_prog_onehot at 0x000002BA8232BF70>


In [6]:
def get_rechtsform_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["rechtsform"], 5).numpy();
    result = pd.DataFrame(tensor, columns=["ref_obl", "ref_fak", "initiative", "gegen_entw", "stichfr"], index=data.index)
    
    return result.astype(int)
print(f"Defined {get_rechtsform_onehot}")

Defined <function get_rechtsform_onehot at 0x000002BA82309EE0>


In [7]:
def get_politikbereich_multihot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    polber = data[["d1e1", "d2e1", "d3e1"]]
    polber = polber.replace('.', 0)
    polber = polber.astype(int)
    
    # the names of the columns (they're a bit long)
    cols = ["Staatsordnung", "Aussenpolitik", "Sicherheitspolitik", "Wirtschaft"]
    cols += ["Landwirtschaft", "Öffentliche Finanzen", "Energie", "Verkehr und Infrastruktur"]
    cols += ["Umwelt und Lebensraum", "Sozialpolitik", "Bildung und Forschung", "Kultur, Religion, Medien"]
    
    result = pd.DataFrame(columns=cols, index = data.index)
    for i in range(len(result)):
        row = np.zeros(12)
        for p in polber.iloc[i]:
            if p != 0:
                row[p-1] = 1
        result.iloc[i] = row
    return result.astype(int)

print(f"Defined {get_politikbereich_multihot}")

Defined <function get_politikbereich_multihot at 0x000002BA84112040>


In [8]:
def get_department_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    dep_single = data["dep"].replace('.', 2) # voting at age 18 is the only vote with a '.' and it's dep of inner
    dep_single = dep_single.astype(int)
    tensor = tf.one_hot(dep_single, 8).numpy()
    result = pd.DataFrame(tensor, columns=["EDA", "EDI", "EJPD", "VBS", "EFD", "WBF", "UVEK", "BK"], index=data.index)
    
    return result.astype(int)

print(f"Defined {get_department_onehot}")

Defined <function get_department_onehot at 0x000002BA82BBEEE0>


In [9]:
def get_bundesrat_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["br_pos"].replace('.', 3).astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_bund", "Dagegen_bund", "Keine_bund"], index=data.index)
    return result.astype(int)

print(f"Defined {get_bundesrat_onehot}")

Defined <function get_bundesrat_onehot at 0x000002BA823CFB80>


In [10]:
def get_legislatur(low:int=1, high:int=10, data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    leg = data["legislatur"]
    def my_map(x:int, x_min:int=leg.min(0), x_max:int=leg.max(0), y_min:int=low, y_max:int=high)->float:
        return (x-x_min)/(x_max-x_min)*(y_max-y_min)+y_min
    
    normalized = data[["legislatur"]].applymap(my_map)
    return normalized

print(f"Defined {get_legislatur}")

Defined <function get_legislatur at 0x000002BA824289D0>


In [11]:
def get_parlament_onehot(data:pd.DataFrame = get_swissvotes_data()["nr_pos"])->pd.DataFrame:
    tensor = tf.one_hot(data.astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_"+data.name, "Dagegen_"+data.name, "Keine_"+data.name], index=data.index)
    return result.astype(int)

print(f"Defined {get_parlament_onehot}")

Defined <function get_parlament_onehot at 0x000002BA8235AE50>


In [12]:
def get_parties(data:pd.DataFrame = get_swissvotes_data())->list:
    import re
    
    regex_incl = re.compile("p_.*")
    regex_excl = re.compile("p_others_.*")
    
    parties_pre = list(filter(regex_incl.match, data.columns))
    parties = [p for p in parties_pre if not regex_excl.match(p)]
    return parties

print(f"Defined {get_parties}")

Defined <function get_parties at 0x000002BA84143700>


In [13]:
def normalize_party_reco(data:pd.DataFrame = get_swissvotes_data(), names:list = get_parties())->pd.DataFrame:
    # deal with unwanted values first
    normalized = data[names].replace(".", 0)
    normalized.replace(np.nan, 0, inplace=True)
    normalized = normalized.astype(int)
    normalized.replace([3,4,5,66,9999], 0, inplace=True)
    
    result = pd.DataFrame(index=normalized.index)
    
    for p in names: # go through parties and create one hot encoding
        tensor = tf.one_hot(normalized[p], 3).numpy()
        temp = pd.DataFrame(tensor, columns=[p+"_neutral", p+"_ja", p+"_nein"], index=result.index)
        result = result.join(temp)

    return result.astype(int)
print(f"Defined {normalize_party_reco}")

Defined <function normalize_party_reco at 0x000002BA825EB0D0>


In [14]:
def get_vote_result(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    result = data["annahme"].replace('.', 0)
    return result.astype(int)

print(f"Defined {get_vote_result}")

Defined <function get_vote_result at 0x000002BA849BCEE0>


In [15]:
# creates a dataframe 
def get_canton_results(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    import re
    regex = re.compile(".*_annahme")
    canton_names = list(filter(regex.match, data.columns))
    return data[canton_names].replace('.', 0).astype(int)

print(f"Defined {get_canton_results}")

Defined <function get_canton_results at 0x000002BA8235CA60>


In [16]:
def get_volk_proz(data:pd.DataFrame=get_swissvotes_data())->pd.DataFrame:
    temp = data["volkja_proz"]
    return temp / 100

print(f"Defined {get_volk_proz}")

Defined <function get_volk_proz at 0x000002BA85A4CEE0>


In [17]:
def get_canton_proz(data:pd.DataFrame=get_swissvotes_data(), name:str=None)->pd.DataFrame:
    if name != None:
        return data[name+"_japroz"].replace('.', 50).astype(float) / 100
    
    import re
    regex = re.compile(".._japroz")
    names = list(filter(regex.match, data.columns))
    return data[names].replace('.', 50).astype(float) / 100

print(f"Defined {get_canton_proz}")

Defined <function get_canton_proz at 0x000002BA84145EE0>


In [18]:
def get_canton_names()->list:
    import re
    regex = re.compile(".._japroz")
    return [x[0:2] for x in list(filter(regex.match, get_swissvotes_data().columns))]

print(f"Defined {get_canton_names}")

Defined <function get_canton_names at 0x000002BA84145550>


## Training the net

In [19]:
# get data    
# the possible inputs for the neural net are:
    # Rechtsform (one hot),
    # Politikbereich (multi hot),
    # Department (one hot),
    # Position of the Bundesrat (one hot),
    # legislatur (normalized from 1-10),
    # Position of Nationalrat (one hot),
    # Position of Ständerat (one hot),
    # Party recommendations (one hot),
    # left, right rating (one hot),
    # conservative, liberal (one hot)
# the possible outputs are:
    # result of the votes (binary),
    # result on a canton level (binary)
    # yes percentage of the people ([0:1]),
    # yes percentage by canton ([0:1])
def get_data(input_func:list=[get_rechtsform_onehot,
                             get_politikbereich_multihot, 
                             get_department_onehot, 
                             get_bundesrat_onehot,
                             get_legislatur,
                             get_parlament_onehot,
                             get_parlament_onehot,
                             normalize_party_reco,
                             get_l_r_onehot,
                             get_kons_prog_onehot], 
            input_args:list=[None, 
                            None,
                            None,
                            None,
                            None,
                            None,
                            get_swissvotes_data()["sr_pos"],
                            None,
                            None,
                            None],
            output_func:list=[get_vote_result,
                             get_canton_results,
                             get_volk_proz, 
                             get_canton_proz],
            output_args:list=[None,
                             None,
                             None,
                             None]):
    input_list = []
    for f, a in zip(input_func, input_args):
        if type(a) == type(None):
            input_list.append(f())
        else:
            input_list.append(f(a))
    
    inputs = pd.concat(input_list, axis=1)
    
    out_list = []
    for f, a in zip(output_func, output_args):
        if type(a) == type(None):
            out_list.append(f())
        else:
            out_list.append(f(a))
    
    outputs = pd.concat(out_list, axis=1)
    
    return get_swissvotes_data(), inputs, outputs

print(f"Defined {get_data}")

Defined <function get_data at 0x000002BA841330D0>


In [20]:
def create_model(name:str, input_size:int = len(get_data()[1].columns), hidden:list=[100, 50, 20],
                 output_size:int = len(get_data(output_func=[get_vote_result,get_canton_results], 
                                                output_args=[None,None])[2].columns), 
                 activation:str="relu", activation_output:str="sigmoid", 
                 optimizer=ks.optimizers.SGD(learning_rate=0.1), 
                 loss=ks.losses.BinaryCrossentropy(), 
                 metrics:list=[ks.metrics.BinaryAccuracy(), ks.metrics.FalseNegatives()])->ks.models.Sequential:
    model = ks.models.Sequential(name=name)
    
    model.add(ks.layers.Dense(units=input_size, activation=activation, name="Input"))
    
    for i in range(len(hidden)):
        model.add(ks.layers.Dense(units=hidden[i], activation=activation, name="Hidden_"+str(i)))
        model.add(ks.layers.Dropout(rate=.1, name="Dropout_"+str(i)))
        
    model.add(ks.layers.Dense(units=output_size, activation=activation_output, name="Output"))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

print(f"Defined {create_model}")

Defined <function create_model at 0x000002BA8235A700>


In [21]:
def train_model(model:ks.models.Sequential, 
                inputs:pd.DataFrame=get_data()[1], 
                outputs:pd.DataFrame=get_data(output_func=[get_vote_result,get_canton_results], 
                                              output_args=[None,None])[2], 
                test_size:float=0.2, batch_size:int=50, epochs:int=125, shuffle:bool=True)->tuple:
    
    from sklearn.model_selection import train_test_split as tss
    in_train, in_test, out_train, out_test = tss(inputs, outputs, test_size=test_size)
    
    history = model.fit(x=in_train, y=out_train, batch_size=batch_size, epochs=epochs, shuffle=shuffle)
    
    return history, in_test, out_test

print(f"Defined {train_model}")

Defined <function train_model at 0x000002BA825EB790>


### Create and Train models

There are two national models. One for binary results and one for exact percentage of yes votes.

In [22]:
optimizer = ks.optimizers.RMSprop(learning_rate=0.01)
epochs = 200

nat_bin = create_model("BinaryModel", optimizer=optimizer, output_size=1, metrics=[ks.metrics.BinaryAccuracy(),ks.metrics.FalseNegatives(),ks.metrics.FalsePositives()])
nat_ext = create_model("ExactModel", optimizer=optimizer, output_size=1, loss=ks.losses.MeanAbsoluteError(), 
                           metrics=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])

history_bin, in_test_bin, out_test_bin = train_model(nat_bin, epochs=epochs,
                                                     outputs=get_data(output_func=[get_vote_result], 
                                                                      output_args=[None])[2])
history_per, in_test_per, out_test_per = train_model(nat_ext, epochs=epochs,
                                                     outputs=get_data(output_func=[get_volk_proz], 
                                                                      output_args=[None])[2])

Epoch 1/200

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

6/6 [==============================] - 0s 834us/step - loss: 1.2904 - binary_accuracy: 0.4688 - false_negatives_1: 78.0000 - false_positives: 75.0000
Epoch 2/200
6/6 [==============================] - 0s 667us/step - loss: 0.6576 - binary_accuracy: 0.6528 - false_negatives_1: 63.0000 - false_positives: 37.0000
Epoch 3/200
6/6 [==============================] - 0s 667us/step - loss: 0.6327 - binary_accuracy: 0.7153 - false_negatives_1: 42.0000 - false_positives: 40.0000
Epoch 4/200
6/6 [=====

6/6 [==============================] - 0s 667us/step - loss: 4.6214e-04 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 47/200
6/6 [==============================] - 0s 834us/step - loss: 1.8728e-04 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 48/200
6/6 [==============================] - 0s 667us/step - loss: 8.1527e-05 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 49/200
6/6 [==============================] - 0s 834us/step - loss: 8.3831e-05 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 50/200
6/6 [==============================] - 0s 667us/step - loss: 1.9849e-04 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 51/200
6/6 [==============================] - 0s 667us/step - loss: 1.2303e-05 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - fal

6/6 [==============================] - 0s 667us/step - loss: 1.4507e-04 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 95/200
6/6 [==============================] - 0s 667us/step - loss: 1.2668e-05 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 96/200
6/6 [==============================] - 0s 667us/step - loss: 4.8194e-05 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 97/200
6/6 [==============================] - 0s 834us/step - loss: 1.0495e-06 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 98/200
6/6 [==============================] - 0s 667us/step - loss: 3.8778e-07 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 99/200
6/6 [==============================] - 0s 667us/step - loss: 0.0260 - binary_accuracy: 0.9965 - false_negatives_1: 1.0000 - false_posit

6/6 [==============================] - 0s 834us/step - loss: 2.1614e-07 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 142/200
6/6 [==============================] - 0s 667us/step - loss: 7.8853e-06 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 143/200
6/6 [==============================] - 0s 667us/step - loss: 3.1033e-06 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 144/200
6/6 [==============================] - 0s 667us/step - loss: 8.6043e-07 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 145/200
6/6 [==============================] - 0s 667us/step - loss: 6.8671e-08 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 146/200
6/6 [==============================] - 0s 667us/step - loss: 4.9341e-07 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 

6/6 [==============================] - 0s 667us/step - loss: 3.6879e-05 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 189/200
6/6 [==============================] - 0s 667us/step - loss: 6.6250e-04 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 190/200
6/6 [==============================] - 0s 667us/step - loss: 8.6839e-07 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 191/200
6/6 [==============================] - 0s 667us/step - loss: 4.5263e-06 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 192/200
6/6 [==============================] - 0s 667us/step - loss: 2.6114e-06 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 - false_positives: 0.0000e+00
Epoch 193/200
6/6 [==============================] - 0s 667us/step - loss: 3.2659e-05 - binary_accuracy: 1.0000 - false_negatives_1: 0.0000e+00 

6/6 [==============================] - 0s 667us/step - loss: 0.0521 - mean_squared_error: 0.0048 - mean_absolute_error: 0.0521
Epoch 39/200
6/6 [==============================] - 0s 667us/step - loss: 0.0590 - mean_squared_error: 0.0062 - mean_absolute_error: 0.0590
Epoch 40/200
6/6 [==============================] - 0s 667us/step - loss: 0.0650 - mean_squared_error: 0.0073 - mean_absolute_error: 0.0650
Epoch 41/200
6/6 [==============================] - 0s 500us/step - loss: 0.0509 - mean_squared_error: 0.0045 - mean_absolute_error: 0.0509
Epoch 42/200
6/6 [==============================] - 0s 667us/step - loss: 0.0536 - mean_squared_error: 0.0048 - mean_absolute_error: 0.0536
Epoch 43/200
6/6 [==============================] - 0s 667us/step - loss: 0.0550 - mean_squared_error: 0.0050 - mean_absolute_error: 0.0550
Epoch 44/200
6/6 [==============================] - 0s 500us/step - loss: 0.0554 - mean_squared_error: 0.0053 - mean_absolute_error: 0.0554
Epoch 45/200
6/6 [===============

6/6 [==============================] - 0s 667us/step - loss: 0.0348 - mean_squared_error: 0.0022 - mean_absolute_error: 0.0348
Epoch 97/200
6/6 [==============================] - 0s 500us/step - loss: 0.0428 - mean_squared_error: 0.0034 - mean_absolute_error: 0.0428
Epoch 98/200
6/6 [==============================] - 0s 667us/step - loss: 0.0299 - mean_squared_error: 0.0015 - mean_absolute_error: 0.0299
Epoch 99/200
6/6 [==============================] - 0s 667us/step - loss: 0.0359 - mean_squared_error: 0.0025 - mean_absolute_error: 0.0359
Epoch 100/200
6/6 [==============================] - 0s 500us/step - loss: 0.0370 - mean_squared_error: 0.0025 - mean_absolute_error: 0.0370
Epoch 101/200
6/6 [==============================] - 0s 667us/step - loss: 0.0385 - mean_squared_error: 0.0025 - mean_absolute_error: 0.0385
Epoch 102/200
6/6 [==============================] - 0s 667us/step - loss: 0.0343 - mean_squared_error: 0.0021 - mean_absolute_error: 0.0343
Epoch 103/200
6/6 [===========

6/6 [==============================] - 0s 500us/step - loss: 0.0281 - mean_squared_error: 0.0015 - mean_absolute_error: 0.0281
Epoch 155/200
6/6 [==============================] - 0s 667us/step - loss: 0.0319 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0319
Epoch 156/200
6/6 [==============================] - 0s 667us/step - loss: 0.0306 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0306
Epoch 157/200
6/6 [==============================] - 0s 667us/step - loss: 0.0301 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0301
Epoch 158/200
6/6 [==============================] - 0s 667us/step - loss: 0.0294 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0294
Epoch 159/200
6/6 [==============================] - 0s 667us/step - loss: 0.0289 - mean_squared_error: 0.0016 - mean_absolute_error: 0.0289
Epoch 160/200
6/6 [==============================] - 0s 667us/step - loss: 0.0319 - mean_squared_error: 0.0019 - mean_absolute_error: 0.0319
Epoch 161/200
6/6 [========

In [23]:
print(nat_bin.summary(), nat_ext.summary())

Model: "BinaryModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (Dense)                (None, 187)               35156     
_________________________________________________________________
Hidden_0 (Dense)             (None, 100)               18800     
_________________________________________________________________
Dropout_0 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_1 (Dense)             (None, 50)                5050      
_________________________________________________________________
Dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
Hidden_2 (Dense)             (None, 20)                1020      
_________________________________________________________________
Dropout_2 (Dropout)          (None, 20)                

In [24]:
res_bin = nat_bin.evaluate(x=in_test_bin, y=out_test_bin)
res_per = nat_ext.evaluate(x=in_test_per, y=out_test_per)

print(len(out_test_bin))
print(f"Binary model results:\t{res_bin}")
print(f"Exact model results:\t{res_per}")

from sklearn.metrics import r2_score
print(f"R2 metric exact:\t{r2_score(nat_ext.predict(in_test_per), out_test_per)}")
print(f"R2 metric binary:\t{r2_score(nat_bin.predict(in_test_bin), out_test_bin)}")

3/3 [==============================] - 0s 667us/step - loss: 0.0880 - mean_squared_error: 0.0115 - mean_absolute_error: 0.0880
72
Binary model results:	[4.070017337799072, 0.875, 8.0, 1.0]
Exact model results:	[0.08804520964622498, 0.011548851616680622, 0.08804520964622498]
R2 metric exact:	0.5933857388418744
R2 metric binary:	0.4765345420322782


**Let's try to create a model for each canton.**

In [25]:
def get_per_canton_models(names:list=get_canton_names(), optimizer=ks.optimizers.SGD(learning_rate=0.01), 
                          loss=ks.losses.MeanAbsoluteError(), 
                          metrics:list=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])->list:
    result = []
    for c in names:
        result.append((create_model("Canton_"+c, output_size=1, 
                                    optimizer=optimizer, loss=loss, 
                                    metrics=metrics), c))
    
    return result
print(f"Defined {get_per_canton_models}")

Defined <function get_per_canton_models at 0x000002BA894EB4C0>


In [26]:
def train_per_canton_models(models:list=get_per_canton_models(), 
                            inputs:pd.DataFrame=get_data()[1], 
                            test_size:float=0.6, batch_size:int=50, epochs:int=125, shuffle:bool=True)->list:
    result = []
    
    from sklearn.model_selection import train_test_split as tss
    for t in models:
        itr, ite, otr, ote = tss(inputs, get_canton_proz(name=t[1]), test_size=test_size)
        result.append((t[0].fit(x=itr, y=otr, batch_size=batch_size, epochs=epochs, shuffle=shuffle), ite, ote))
        
    return result
        
print(f"Defined {train_per_canton_models}")

Defined <function train_per_canton_models at 0x000002BA894AD9D0>


In [27]:
def execute_models():
    models = get_per_canton_models()
    hist = train_per_canton_models(models, epochs=150)
    
    evals = []
    for m, h in zip(models, hist):
        evals.append(m[0].evaluate(x=h[1], y=h[2]))
    
    for m, e in zip(models, evals):
        print(f"{m[0].name} has these results:\t{e}")
        
    return models, evals

In [28]:
def print_extreme_cases():
    models, evals = execute_models()
    
    min_abs = ("", 1)
    max_abs = ("", 0)
    min_sqr = ("", 1)
    max_sqr = ("", 0)
    
    for m, e in zip(models, evals):
        if(e[2]<min_abs[1]):
            min_abs = (m[0].name, e[2])
        elif(e[2]>max_abs[1]):
            max_abs = (m[0].name, e[2])
    
        if(e[1]<min_sqr[1]):
            min_sqr = (m[0].name, e[1])
        elif(e[1]>max_sqr[1]):
            max_sqr = (m[0].name, e[1])
        
    print(f"{max_sqr[0]} has the largest mse of\t{max_sqr[1]}")
    print(f"{min_sqr[0]} has the smallest mse of\t{min_sqr[1]}")
    print(f"{max_abs[0]} has the largest mae of\t{max_abs[1]}")
    print(f"{min_abs[0]} has the smallest mae of\t{min_abs[1]}")

In [29]:
print_extreme_cases()

Epoch 1/150

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

3/3 [==============================] - 0s 667us/step - loss: 0.1791 - mean_squared_error: 0.0510 - mean_absolute_error: 0.1791
Epoch 2/150
3/3 [==============================] - 0s 667us/step - loss: 0.1721 - mean_squared_error: 0.0470 - mean_absolute_error: 0.1721
Epoch 3/150
3/3 [==============================] - 0s 334us/step - loss: 0.1596 - mean_squared_error: 0.0403 - mean_absolute_error: 0.1596
Epoch 4/150
3/3 [==============================] - 0s 667us/step - loss: 0.1661 - mean_squa

3/3 [==============================] - 0s 334us/step - loss: 0.1202 - mean_squared_error: 0.0221 - mean_absolute_error: 0.1202
Epoch 55/150
3/3 [==============================] - 0s 667us/step - loss: 0.1151 - mean_squared_error: 0.0208 - mean_absolute_error: 0.1151
Epoch 56/150
3/3 [==============================] - 0s 334us/step - loss: 0.1172 - mean_squared_error: 0.0226 - mean_absolute_error: 0.1172
Epoch 57/150
3/3 [==============================] - 0s 667us/step - loss: 0.1173 - mean_squared_error: 0.0221 - mean_absolute_error: 0.1173
Epoch 58/150
3/3 [==============================] - 0s 667us/step - loss: 0.1151 - mean_squared_error: 0.0216 - mean_absolute_error: 0.1151
Epoch 59/150
3/3 [==============================] - 0s 667us/step - loss: 0.1167 - mean_squared_error: 0.0223 - mean_absolute_error: 0.1167
Epoch 60/150
3/3 [==============================] - 0s 667us/step - loss: 0.1160 - mean_squared_error: 0.0213 - mean_absolute_error: 0.1160
Epoch 61/150
3/3 [===============

3/3 [==============================] - 0s 334us/step - loss: 0.1003 - mean_squared_error: 0.0174 - mean_absolute_error: 0.1003
Epoch 113/150
3/3 [==============================] - 0s 667us/step - loss: 0.1024 - mean_squared_error: 0.0169 - mean_absolute_error: 0.1024
Epoch 114/150
3/3 [==============================] - 0s 333us/step - loss: 0.0995 - mean_squared_error: 0.0162 - mean_absolute_error: 0.0995
Epoch 115/150
3/3 [==============================] - 0s 667us/step - loss: 0.0984 - mean_squared_error: 0.0162 - mean_absolute_error: 0.0984
Epoch 116/150
3/3 [==============================] - 0s 667us/step - loss: 0.0962 - mean_squared_error: 0.0149 - mean_absolute_error: 0.0962
Epoch 117/150
3/3 [==============================] - 0s 334us/step - loss: 0.1015 - mean_squared_error: 0.0168 - mean_absolute_error: 0.1015
Epoch 118/150
3/3 [==============================] - 0s 667us/step - loss: 0.1009 - mean_squared_error: 0.0162 - mean_absolute_error: 0.1009
Epoch 119/150
3/3 [========

3/3 [==============================] - 0s 333us/step - loss: 0.1489 - mean_squared_error: 0.0338 - mean_absolute_error: 0.1489
Epoch 14/150
3/3 [==============================] - 0s 667us/step - loss: 0.1500 - mean_squared_error: 0.0330 - mean_absolute_error: 0.1500
Epoch 15/150
3/3 [==============================] - 0s 334us/step - loss: 0.1539 - mean_squared_error: 0.0344 - mean_absolute_error: 0.1539
Epoch 16/150
3/3 [==============================] - 0s 667us/step - loss: 0.1534 - mean_squared_error: 0.0353 - mean_absolute_error: 0.1534
Epoch 17/150
3/3 [==============================] - 0s 667us/step - loss: 0.1502 - mean_squared_error: 0.0336 - mean_absolute_error: 0.1502
Epoch 18/150
3/3 [==============================] - 0s 667us/step - loss: 0.1492 - mean_squared_error: 0.0332 - mean_absolute_error: 0.1492
Epoch 19/150
3/3 [==============================] - 0s 667us/step - loss: 0.1505 - mean_squared_error: 0.0328 - mean_absolute_error: 0.1505
Epoch 20/150
3/3 [===============

3/3 [==============================] - 0s 667us/step - loss: 0.1212 - mean_squared_error: 0.0231 - mean_absolute_error: 0.1212
Epoch 72/150
3/3 [==============================] - 0s 334us/step - loss: 0.1268 - mean_squared_error: 0.0254 - mean_absolute_error: 0.1268
Epoch 73/150
3/3 [==============================] - 0s 667us/step - loss: 0.1262 - mean_squared_error: 0.0242 - mean_absolute_error: 0.1262
Epoch 74/150
3/3 [==============================] - 0s 667us/step - loss: 0.1216 - mean_squared_error: 0.0232 - mean_absolute_error: 0.1216
Epoch 75/150
3/3 [==============================] - 0s 667us/step - loss: 0.1285 - mean_squared_error: 0.0250 - mean_absolute_error: 0.1285
Epoch 76/150
3/3 [==============================] - 0s 667us/step - loss: 0.1289 - mean_squared_error: 0.0251 - mean_absolute_error: 0.1289
Epoch 77/150
3/3 [==============================] - 0s 333us/step - loss: 0.1276 - mean_squared_error: 0.0249 - mean_absolute_error: 0.1276
Epoch 78/150
3/3 [===============

...

3/3 [==============================] - 0s 667us/step - loss: 0.1540 - mean_squared_error: 0.0350 - mean_absolute_error: 0.1540
Epoch 43/150
3/3 [==============================] - 0s 334us/step - loss: 0.1546 - mean_squared_error: 0.0343 - mean_absolute_error: 0.1546
Epoch 44/150
3/3 [==============================] - 0s 667us/step - loss: 0.1496 - mean_squared_error: 0.0330 - mean_absolute_error: 0.1496
Epoch 45/150
3/3 [==============================] - 0s 334us/step - loss: 0.1499 - mean_squared_error: 0.0318 - mean_absolute_error: 0.1499
Epoch 46/150
3/3 [==============================] - 0s 667us/step - loss: 0.1536 - mean_squared_error: 0.0337 - mean_absolute_error: 0.1536
Epoch 47/150
3/3 [==============================] - 0s 667us/step - loss: 0.1595 - mean_squared_error: 0.0366 - mean_absolute_error: 0.1595
Epoch 48/150
3/3 [==============================] - 0s 667us/step - loss: 0.1542 - mean_squared_error: 0.0350 - mean_absolute_error: 0.1542
Epoch 49/150
3/3 [===============

3/3 [==============================] - 0s 667us/step - loss: 0.1330 - mean_squared_error: 0.0263 - mean_absolute_error: 0.1330
Epoch 101/150
3/3 [==============================] - 0s 334us/step - loss: 0.1332 - mean_squared_error: 0.0260 - mean_absolute_error: 0.1332
Epoch 102/150
3/3 [==============================] - 0s 667us/step - loss: 0.1418 - mean_squared_error: 0.0289 - mean_absolute_error: 0.1418
Epoch 103/150
3/3 [==============================] - 0s 667us/step - loss: 0.1354 - mean_squared_error: 0.0266 - mean_absolute_error: 0.1354
Epoch 104/150
3/3 [==============================] - 0s 667us/step - loss: 0.1258 - mean_squared_error: 0.0240 - mean_absolute_error: 0.1258
Epoch 105/150
3/3 [==============================] - 0s 333us/step - loss: 0.1252 - mean_squared_error: 0.0225 - mean_absolute_error: 0.1252
Epoch 106/150
3/3 [==============================] - 0s 667us/step - loss: 0.1270 - mean_squared_error: 0.0260 - mean_absolute_error: 0.1270
Epoch 107/150
3/3 [========

3/3 [==============================] - 0s 667us/step - loss: 0.1729 - mean_squared_error: 0.0443 - mean_absolute_error: 0.1729
Epoch 4/150
3/3 [==============================] - 0s 667us/step - loss: 0.1781 - mean_squared_error: 0.0452 - mean_absolute_error: 0.1781
Epoch 5/150
3/3 [==============================] - 0s 667us/step - loss: 0.1728 - mean_squared_error: 0.0433 - mean_absolute_error: 0.1728
Epoch 6/150
3/3 [==============================] - 0s 334us/step - loss: 0.1633 - mean_squared_error: 0.0391 - mean_absolute_error: 0.1633
Epoch 7/150
3/3 [==============================] - 0s 667us/step - loss: 0.1682 - mean_squared_error: 0.0430 - mean_absolute_error: 0.1682
Epoch 8/150
3/3 [==============================] - 0s 667us/step - loss: 0.1682 - mean_squared_error: 0.0419 - mean_absolute_error: 0.1682
Epoch 9/150
3/3 [==============================] - 0s 668us/step - loss: 0.1611 - mean_squared_error: 0.0397 - mean_absolute_error: 0.1611
Epoch 10/150
3/3 [=====================

3/3 [==============================] - 0s 333us/step - loss: 0.1382 - mean_squared_error: 0.0290 - mean_absolute_error: 0.1382
Epoch 62/150
3/3 [==============================] - 0s 668us/step - loss: 0.1372 - mean_squared_error: 0.0292 - mean_absolute_error: 0.1372
Epoch 63/150
3/3 [==============================] - 0s 333us/step - loss: 0.1359 - mean_squared_error: 0.0282 - mean_absolute_error: 0.1359
Epoch 64/150
3/3 [==============================] - 0s 667us/step - loss: 0.1304 - mean_squared_error: 0.0272 - mean_absolute_error: 0.1304
Epoch 65/150
3/3 [==============================] - 0s 334us/step - loss: 0.1357 - mean_squared_error: 0.0289 - mean_absolute_error: 0.1357
Epoch 66/150
3/3 [==============================] - 0s 667us/step - loss: 0.1297 - mean_squared_error: 0.0267 - mean_absolute_error: 0.1297
Epoch 67/150
3/3 [==============================] - 0s 334us/step - loss: 0.1308 - mean_squared_error: 0.0265 - mean_absolute_error: 0.1308
Epoch 68/150
3/3 [===============

3/3 [==============================] - 0s 667us/step - loss: 0.1202 - mean_squared_error: 0.0214 - mean_absolute_error: 0.1202
Epoch 120/150
3/3 [==============================] - 0s 667us/step - loss: 0.1134 - mean_squared_error: 0.0201 - mean_absolute_error: 0.1134
Epoch 121/150
3/3 [==============================] - 0s 667us/step - loss: 0.1103 - mean_squared_error: 0.0193 - mean_absolute_error: 0.1103
Epoch 122/150
3/3 [==============================] - 0s 667us/step - loss: 0.1152 - mean_squared_error: 0.0204 - mean_absolute_error: 0.1152
Epoch 123/150
3/3 [==============================] - 0s 667us/step - loss: 0.1133 - mean_squared_error: 0.0194 - mean_absolute_error: 0.1133
Epoch 124/150
3/3 [==============================] - 0s 667us/step - loss: 0.1225 - mean_squared_error: 0.0223 - mean_absolute_error: 0.1225
Epoch 125/150
3/3 [==============================] - 0s 667us/step - loss: 0.1120 - mean_squared_error: 0.0196 - mean_absolute_error: 0.1120
Epoch 126/150
3/3 [========